# DATA CLEANING (TECHNICAL CODING ROUND)
__ Gahan Jagadeesh __

__ Email : gj717@nyu.edu __

__ Ph : (917) - 349 - 3801 __

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

### __IMPORT THE DATASET__ 

In [2]:
df = pd.read_csv('BL-Flickr-Images-Book.csv')

#### __READING THE FIRST FEW ENTRIES OF THE DATASET__

In [3]:
df.head(5)

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


## DROPPING UNNECESSARY COLUMNS
__ Most of the large datasets contain a lot of unnecessary columns which are not useful for data analysis . Such columns are deemed unnecessary based on the intuition of a user. I find some columns unnecessary if they are not populated as much as the rest of the columns, or if they have a lot of NaN values which would cause a lot of problems when we are applying ML algorithms to it. 
Let us begin by dropping some unnecessary columns.__

In [4]:
to_drop = ['Edition Statement',
           'Corporate Author',
           'Corporate Contributors',
           'Former owner',
           'Engraver',
           'Contributors',
           'Issuance type',
           'Shelfmarks']

df.drop(to_drop, inplace = True, axis = 1)
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


In [5]:
df.dropna(subset=['Author'],inplace = True)

## Cleaning the messy columns

 __Some data entries are really messy and needs to be cleaned as they contain several values which makes it hard to analyse the data such as [,],:,.,'," etc __

__ The Date of Publication column is slightly messy as it has some values such as ' [ ', ' ] ', '-', '?' etc. It would be better if we only had years in the column. Let us see how we can clean it __

In [6]:
unwanted_characters = ['[', ',', '-',']']

def clean_dates(item):
    dop= str(item.loc['Date of Publication'])
    
    if dop == 'nan' or dop[0] == '[':
        return np.NaN
    
    for character in unwanted_characters:
        if character in dop:
            character_index = dop.find(character)
            dop = dop[:character_index]
    
    return dop

df['Date of Publication'] = df.apply(clean_dates, axis = 1)

In [7]:
df['Date of Publication'].head(10)

0    1879 
1     1868
2     1869
3     1851
4     1857
5     1875
6     1872
7      NaN
8     1676
9     1679
Name: Date of Publication, dtype: object

__ The next such column is the 'Title' column. Let us try to clean that column now. __

In [8]:
def clean_title(title):
    
    if title == 'nan':
        return 'NaN'
    
    if title[0] == '[':
        title = title[1: title.find(']')]
        
    if 'by' in title:
        title = title[:title.find('by')]
    elif 'By' in title:
        title = title[:title.find('By')]
        
    if '[' in title:
        title = title[:title.find('[')]

    title = title[:-2]
        
    title = list(map(str.capitalize, title.split()))
    return ' '.join(title)
    
df['Title'] = df['Title'].apply(clean_title)
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879,S. Tinsley & Co.,Walter Forbes,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All For Greed,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love The Avenger,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, Chiefly Ecclesiastical, To The...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"The World In Which I Live, And My Place In It","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


__ The 'Author' column needs to be tweaked a little too __

In [9]:
def clean_author_names(author):
    
    author = str(author)
    
    if author == 'nan':
        return 'NaN'
    
    author = author.split(',')

    if len(author) == 1:
        name = filter(lambda x: x.isalpha(), author[0])
        return reduce(lambda x, y: x + y, name)
    
    last_name, first_name = author[0], author[1]

    first_name = first_name[:first_name.find('-')] if '-' in first_name else first_name
    
    if first_name.endswith(('.', '.|')):
        parts = first_name.split('.')
        
        if len(parts) > 1:
            first_occurence = first_name.find('.')
            final_occurence = first_name.find('.', first_occurence + 1)
            first_name = first_name[:final_occurence]
        else:
            first_name = first_name[:first_name.find('.')]
    
    last_name = last_name.capitalize()
    
    return f'{first_name} {last_name}'


df['Author'] = df['Author'].apply(clean_author_names)

df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879,S. Tinsley & Co.,Walter Forbes,AA,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All For Greed,A. A A.,http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love The Avenger,A. A A.,http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, Chiefly Ecclesiastical, To The...",E. S A.,http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"The World In Which I Live, And My Place In It",E. S A.,http://www.flickr.com/photos/britishlibrary/ta...


__ The 'Place of publication' has a few complications too. Especially with places containing London and Oxford . Let us fix it using numpy's if() method __

In [10]:
pub=df['Place of Publication']
df['Place of Publication'] = np.where(pub.str.contains('London'), 'London',
                                      np.where(pub.str.contains('Oxford'), 'Oxford',
                                               pub.str.replace('-', ' ')))

In [11]:
df['Place of Publication'].head(15)

0          London
1          London
2          London
3          London
4          London
5          London
6          London
7          Oxford
8          London
9          London
11    Christiania
12        Firenze
14         Savona
15         London
16          Paris
Name: Place of Publication, dtype: object

__ As we can see in the above cell, We have corrected the Place of Publication column
Let us take a look at the entire head of the table now __

In [12]:
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879,S. Tinsley & Co.,Walter Forbes,AA,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London,1868,Virtue & Co.,All For Greed,A. A A.,http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love The Avenger,A. A A.,http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, Chiefly Ecclesiastical, To The...",E. S A.,http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"The World In Which I Live, And My Place In It",E. S A.,http://www.flickr.com/photos/britishlibrary/ta...


### Detecting missing values and dealing with them
__The missing values could be in many ways. Most common ways of missing values are the NaN values or nothing at all being present in the entries. Let us first look at what to do with the NaN values__

__Let us first count the number of values which are NaN__

In [13]:
df.isnull().sum(axis=0)

Identifier                 0
Place of Publication       0
Date of Publication      724
Publisher               3445
Title                      0
Author                     0
Flickr URL                 0
dtype: int64

__ The Publisher column has 4195 Missing values, So it is better for us to discard it completely __

In [14]:
df.drop('Publisher', inplace = True, axis = 1)

__ The publisher column has been dropped __

__ The rows which do not contain the dates of publication will be dropped mainly for one reason, The dates of publication are important for the data analysis and it does not make sense to continue with missing dates. Usually, for missing values -- WE INPUT THE MEAN OR MEDIAN DEPENDING ON THE SITUATION. But here, we have not done that because taking the mean and median of a date is meaningless and will not give meaningful results. __

In [15]:
df.dropna(subset=['Date of Publication'],inplace=True)

### The messy data has been cleaned with all the missing values eliminated.

__ Now, the head of the dataset will be __

In [18]:
df.head()

,Identifier,Place of Publication,Date of Publication,Title,Author,Flickr URL
0,206,London,1879,Walter Forbes,AA,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London,1868,All For Greed,A. A A.,http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,Love The Avenger,A. A A.,http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,"Welsh Sketches, Chiefly Ecclesiastical, To The...",E. S A.,http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,"The World In Which I Live, And My Place In It",E. S A.,http://www.flickr.com/photos/britishlibrary/ta...


In [23]:
df.isnull().sum()

Identifier              0
Place of Publication    0
Date of Publication     0
Title                   0
Author                  0
Flickr URL              0
dtype: int64

__ Based on the above information, we now do not have any missing values in the dataset __